In [1]:
import pandas as pd
from tqdm import tqdm
import json, os

Not much overlap between testis images in HPAv21 and the test set of Ghoshal et al (HPAv18/19).

In [2]:
hpa = pd.read_csv('./data/hpa_v21.csv.gz',index_col=0).query('Tissue=="testis"')

In [3]:
df = pd.read_excel('./data/mmc1.xlsx',skiprows=2,index_col=0)
df.to_csv('./data/testis_labels.csv')

In [4]:
df = pd.read_excel('./data/mmc1.xlsx',skiprows=2,index_col=0)
urls = '\n'.join('http:' + df['Image_URL'])
!mkdir -p ./data/images/testis
%store urls >./data/images/testis/urls.txt

Writing 'urls' (str) to file './data/images/testis/urls.txt'.


In [5]:
# !cd ./data/images/testis && wget -c -x -t3 -nH -nc -i urls.txt

In [6]:
TISSUE = 'testis'

df = pd.read_excel('./data/mmc1.xlsx',skiprows=2,index_col=0)
df = df.reset_index()[['ENSG_ID','Antibody_ID','Gene name','Image_URL']]
df.columns = ['Gene','Antibody','Gene name','URL']
df['URL'] = 'http:' + df['URL']
df[['Batch','Slide','Well']] = df['URL'].str.extract(r'(\d+)/(\d+)_([A-Z]+_\d+_\d+).jpg')
df['Image'] = df['Slide'] + '_' + df['Well']

df['Staining'] = float('nan')
df['Reliability'] = float('nan')
df['UniProt'] = float('nan')
df['Tissue'] = 'testis'
df['Sex'] = 'Male'
df['Age'] = -1
df['Patient'] = -1
df['nTPM'] = -1

df

,Gene,Antibody,Gene name,URL,Batch,Slide,Well,Image,Staining,Reliability,UniProt,Tissue,Sex,Age,Patient,nTPM
0,ENSG00000000457,HPA005624,SCYL3,http://images.proteinatlas.org/5624/17921_A_5_...,5624,17921,A_5_6,17921_A_5_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
1,ENSG00000000457,HPA005624,SCYL3,http://images.proteinatlas.org/5624/17921_A_6_...,5624,17921,A_6_6,17921_A_6_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
2,ENSG00000000457,HPA005624,SCYL3,http://images.proteinatlas.org/5624/17921_A_4_...,5624,17921,A_4_6,17921_A_4_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
3,ENSG00000004961,HPA002946,HCCS,http://images.proteinatlas.org/2946/8261_A_5_6...,2946,8261,A_5_6,8261_A_5_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
4,ENSG00000004961,HPA002946,HCCS,http://images.proteinatlas.org/2946/8261_A_6_6...,2946,8261,A_6_6,8261_A_6_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369,ENSG00000275722,HPA053073,LYZL6,http://images.proteinatlas.org/53073/145587_A_...,53073,145587,A_6_6,145587_A_6_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
1370,ENSG00000275722,HPA053073,LYZL6,http://images.proteinatlas.org/53073/145587_A_...,53073,145587,A_4_6,145587_A_4_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
1371,ENSG00000284862,HPA035364,CCDC39,http://images.proteinatlas.org/35364/140656_A_...,35364,140656,A_5_6,140656_A_5_6,NaN,NaN,NaN,testis,Male,-1,-1,-1
1372,ENSG00000284862,HPA035364,CCDC39,http://images.proteinatlas.org/35364/140656_A_...,35364,140656,A_6_6,140656_A_6_6,NaN,NaN,NaN,testis,Male,-1,-1,-1


In [ ]:
for _, row in tqdm(df.reset_index().iterrows(),position=0):
    image = row['Image']
    batch = row['Batch']
    url = row['URL']
    path = f'./data/images/{TISSUE}/{batch}'
    !mkdir -p {path}
    if not os.path.exists(f'{path}/{image}.jpg'): 
        continue
    !mogrify -extent 3072x3072 -gravity center -scale 512x512 -format png8 {path}/{image}.jpg
    !rm {path}/{image}.jpg
    !mv {path}/{image}.png8 {path}/{image}.png
    with open(f'{path}/{image}.json','w') as f:
        json.dump(row.to_dict(), f)

In [8]:
# merge the two
df = df.set_index('Image')
df = pd.concat([hpa,df])
df.to_csv('./data/hpa_v21_testis.csv')
df

,Gene name,Gene,UniProt,Antibody,Tissue,Staining,Sex,Age,Patient,URL,Batch,Slide,Well,Reliability,nTPM
Image,,,,,,,,,,,,,,,
11813_A_4_6,TSPAN6,ENSG00000000003,O43657,HPA004109,testis,low,Male,26.0,2071,http://images.proteinatlas.org/4109/11813_A_4_...,4109,11813,A_4_6,Approved,75.5
11813_A_5_6,TSPAN6,ENSG00000000003,O43657,HPA004109,testis,low,Male,68.0,1880,http://images.proteinatlas.org/4109/11813_A_5_...,4109,11813,A_5_6,Approved,75.5
11813_A_6_6,TSPAN6,ENSG00000000003,O43657,HPA004109,testis,low,Male,46.0,2435,http://images.proteinatlas.org/4109/11813_A_6_...,4109,11813,A_6_6,Approved,75.5
132061_A_4_6,TNMD,ENSG00000000005,Q9H2S6,HPA034961,testis,medium,Male,13.0,2037,http://images.proteinatlas.org/34961/132061_A_...,34961,132061,A_4_6,NaN,0.1
132061_A_5_6,TNMD,ENSG00000000005,Q9H2S6,HPA034961,testis,medium,Male,25.0,3627,http://images.proteinatlas.org/34961/132061_A_...,34961,132061,A_5_6,NaN,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145587_A_6_6,LYZL6,ENSG00000275722,NaN,HPA053073,testis,NaN,Male,-1.0,-1,http://images.proteinatlas.org/53073/145587_A_...,53073,145587,A_6_6,NaN,-1.0
145587_A_4_6,LYZL6,ENSG00000275722,NaN,HPA053073,testis,NaN,Male,-1.0,-1,http://images.proteinatlas.org/53073/145587_A_...,53073,145587,A_4_6,NaN,-1.0
140656_A_5_6,CCDC39,ENSG00000284862,NaN,HPA035364,testis,NaN,Male,-1.0,-1,http://images.proteinatlas.org/35364/140656_A_...,35364,140656,A_5_6,NaN,-1.0
